<a href="https://colab.research.google.com/github/mahima-c/Fake-New-Prediction-LSTM/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! ls

gdrive	sample_data


In [ ]:
%cd gdrive/My Drive/project_folder/Fake-New-Prediction-LSTM/


[Errno 2] No such file or directory: 'gdrive/My Drive/project_folder/Fake-New-Prediction-LSTM/'
/content/gdrive/My Drive/project_folder


In [ ]:
pwd

'/content/gdrive/My Drive/project_folder'

In [ ]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
df.shape

(18285, 5)

In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
import tensorflow as tf


In [ ]:
tf.__version__


'2.4.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
### Vocabulary size
voc_size=5000

Onehot **Representation**


In [ ]:
messages=X.copy()

In [ ]:
messages['title'].head(10)
messages.index
if 6 not in messages.index:
  print("4")


4


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# # dataset preprocessing
# from nltk.stem.porter import PorterStemmer
# ps = PorterStemmer()
# corpus = []
# for i in range(0, len(messages)):
#   if i in messages.index:
#     print(i)
#     review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
#     review = review.lower()
#     review = review.split()
    
#     review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
#     review = ' '.join(review)
#     corpus.append(review)

In [ ]:
# dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for sents in messages['title']:
  # if i in messages.index:
    # print(i)
    review = re.sub('[^a-zA-Z]', ' ', sents)
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
len(corpus)

18285

In [ ]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
one_hot(corpus[1],1500)

[117, 528, 370, 531, 711, 344, 1471]

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [ ]:
onehot_repr=[one_hot(sents,voc_size)for sents in corpus] 
onehot_repr

[[216, 4593, 3696, 786, 3949, 1764, 4235, 4573, 3808, 602],
 [194, 415, 131, 2935, 1978, 4008, 1908],
 [3398, 4603, 3376, 1763],
 [3730, 1201, 2737, 3776, 3477, 4702],
 [205, 1978, 277, 10, 1976, 48, 1978, 4734, 240, 3799],
 [3975,
  3355,
  1181,
  4516,
  680,
  3828,
  3507,
  1010,
  1090,
  2300,
  3389,
  1773,
  1355,
  3329,
  1908],
 [420, 3074, 2223, 2719, 140, 927, 1956, 4190, 2221, 1475, 2590],
 [1572, 2580, 506, 48, 4164, 3544, 3828, 4529, 2221, 1475, 2590],
 [2307, 2607, 2157, 2813, 1099, 3687, 2752, 3558, 3828, 1034],
 [4553, 2866, 672, 2603, 861, 4857, 3928, 2195],
 [2611, 4074, 690, 1343, 4694, 4153, 673, 401, 4866, 3269, 2773],
 [3776, 920, 3949, 3687, 3828, 4164],
 [720, 4346, 2643, 155, 236, 1350, 2395, 1640, 273],
 [220, 1152, 3670, 294, 162, 4378, 134, 2221, 1475, 2590],
 [805, 2742, 3310, 2089, 3490, 2221, 1475, 2590],
 [4649, 2812, 575, 3174, 1354, 3405, 2093, 4890, 3153, 2721],
 [187, 1369, 415],
 [999, 3014, 4569, 1347, 3828, 508, 855, 1908],
 [2296, 2044, 131

Embedding Representation


In [ ]:
# first find max length
def findMaxLength(onehot_repr):
  max=0
  for item in onehot_repr:
    # print(len(item))
    if len(item) > max:
      max=len(item)
  return max

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)


In [ ]:
embedded_docs[0]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  216, 4593, 3696,  786, 3949, 1764, 4235,
       4573, 3808,  602], dtype=int32)

In [ ]:
## Creating mode
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.3))

model.add(Dense(1,activation='sigmoid'))
model.add(Dropout(0.3))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
_________________________________________________________________
dropout_5 (Dropout)          (None, 1)                 0         
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
# Performance Metrics And Accuracy
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[ 266, 3153],
       [ 314, 2302]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4255178127589064